In [8]:
import pandas as pd
import glob
from opencc import OpenCC
import ast
import re
import os

In [9]:
print(glob.glob("./xlsx/*.xlsx"))

['./xlsx/欠操.xlsx', './xlsx/機掰.xlsx', './xlsx/黑鬼.xlsx', './xlsx/腦破.xlsx', './xlsx/雞掰.xlsx', './xlsx/廢物.xlsx', './xlsx/婊子.xlsx', './xlsx/畜生.xlsx', './xlsx/垃圾.xlsx', './xlsx/恐龍妹.xlsx', './xlsx/支畜.xlsx', './xlsx/肥宅.xlsx', './xlsx/精障.xlsx', './xlsx/醜男.xlsx', './xlsx/欠揍.xlsx', './xlsx/弱智.xlsx', './xlsx/腦殘.xlsx', './xlsx/破腦.xlsx', './xlsx/欠幹.xlsx', './xlsx/噁男.xlsx', './xlsx/白痴.xlsx', './xlsx/幹你.xlsx', './xlsx/死胖.xlsx', './xlsx/變態.xlsx', './xlsx/破麻.xlsx', './xlsx/神經病.xlsx', './xlsx/肥豬.xlsx', './xlsx/低能.xlsx', './xlsx/智障.xlsx', './xlsx/欠扁.xlsx', './xlsx/喜憨兒.xlsx', './xlsx/白目.xlsx', './xlsx/遲緩.xlsx', './xlsx/人妖.xlsx', './xlsx/娘炮.xlsx', './xlsx/母豬.xlsx', './xlsx/無腦.xlsx', './xlsx/操你.xlsx', './xlsx/臭甲.xlsx', './xlsx/啟智.xlsx', './xlsx/番仔.xlsx', './xlsx/臭婊.xlsx', './xlsx/港仔.xlsx', './xlsx/醜女.xlsx', './xlsx/老處女.xlsx', './xlsx/黑奴.xlsx', './xlsx/屁孩.xlsx', './xlsx/妓女.xlsx', './xlsx/怪胎.xlsx']


In [10]:
xlsx_list = glob.glob("./xlsx/*.xlsx")

In [11]:
print(os.path.splitext(xlsx_list[0])[0])

./xlsx/欠操


In [19]:
print(os.path.splitext(xlsx_list[0])[0].rsplit('/', 2)[2])

欠操


In [20]:
def filter_emoji(desstr, restr=''):
    try:
        co = re.compile(u'[\U00010000-\U0010ffff]')
    except:
        co = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
    return co.sub(restr, desstr)

In [25]:
for xlsx in xlsx_list:
    df = pd.read_excel(xlsx, usecols=["Post Description","Comments"])

    comments = df["Comments"]
    comments_dicts = comments.to_dict()

    posts_list = []

    cc = OpenCC('s2tw')

    for key, value in comments_dicts.items():
        val_str = comments_dicts[key]
        try:
            val_dict = ast.literal_eval(val_str)
        except SyntaxError:
            
            try:
                index_list = [pos for pos, char in enumerate(val_str) if char == ',']
                new_val_str = val_str[:index_list[-2]] + "}"
                val_dict = ast.literal_eval(new_val_str)
            except SyntaxError:
                new_val_str = val_str[:index_list[-3]] + "}"
                val_dict = ast.literal_eval(new_val_str)
        except ValueError:
            val_str = "{}"
        post_list = list(val_dict.values())
        for index, item in enumerate(post_list):
            post_list[index] = filter_emoji(post_list[index])
            post_list[index] = re.sub(r'(http|https)?:\/\/\S*', '', post_list[index], flags=re.MULTILINE)
            post_list[index] = re.sub(r'[0-9\t\n\s+-—•·.。⋯…～~”“？?！!=「」『』（）()《》【】\u200d♀️♂☜☞⚠#♥❤╰⊙╮◉‿❗✊]+', '', post_list[index])
            if post_list[index].encode('utf-8').isalpha():
                post_list[index] = ""
            if len(post_list[index]) < 4:
                post_list[index] = ""
            elif len(post_list[index]) > 30:
                post_list[index] = ""
            post_list[index] = cc.convert(post_list[index])
        post_list = list(filter(None, post_list))
        posts_list.extend(post_list)
    
    filename = os.path.splitext(xlsx)[0].rsplit('/', 2)[2]
    
    textfile = open("./text/"+filename+".txt","w")
    for element in posts_list:
        textfile.write(element + '\n')
    textfile.close()
    